# Imports

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, Expression
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME

import pickle

import numpy as np
import os

from scipy.stats import chi2

# Data & Variables

In [2]:
# Define the relative path to the data folder
file_path = os.path.join(os.pardir, 'lpmc01.dat')

#file_path = os.path.join(data_folder, 'lpmc01.dat')

df = pd.read_csv(file_path, sep = '\t')
df['age_normalized'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['age_scaled'] = df['age'] / df['age'].max()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

columns_to_remove = ['cost_driving_ccharge', 'cost_driving_fuel', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_int', 'dur_pt_bus']
df = df.drop(columns=columns_to_remove)


database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
driving_traffic_percent = Variable('driving_traffic_percent')


variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


# Model Definition

In [3]:
# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

In [4]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

B_TIME_WALK_AGE = Beta('B_TIME_WALK_AGE', 0, None, None, 0)
B_TIME_CYCLE_AGE = Beta('B_TIME_CYCLE_AGE', 0, None, None, 0) 
B_TIME_PT_AGE = Beta('B_TIME_PT_AGE', 0, None, None, 0)
B_TIME_DRIVE_AGE = Beta('B_TIME_DRIVE_AGE', 0, None, None, 0) * age_scaled

# Updated utility functions with age interaction for travel time
V_WALK = (B_TIME_WALK + B_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + B_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + B_COST * cost_transit + (B_TIME_PT + B_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE + B_COST * cost_driving + (B_TIME_DRIVE + B_TIME_DRIVE_AGE * age_scaled) * dur_driving

V = {1: V_WALK, 2: V_CYCLE, 3: V_PT, 4: V_DRIVE}


logprob_m4 = loglogit(V, availability, travel_mode)

model_4 = bio.BIOGEME(database, logprob_m4)
model_4.modelName = 'model_4'

results_m4 = model_4.estimate()
model_4_loglike = results_m4.data.logLike
model_4_numParam = results_m4.get_estimated_parameters().shape[0]
model_4_table = results_m4.get_estimated_parameters()

model_4_table

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CYCLE,-4.622020,0.201107,-22.982863,0.000000e+00
ASC_DRIVE,-2.152861,0.149686,-14.382526,0.000000e+00
ASC_PT,-2.615048,0.150215,-17.408726,0.000000e+00
B_COST,-0.179551,0.017618,-10.191389,0.000000e+00
B_TIME_CYCLE,-4.646150,0.755998,-6.145715,7.960415e-10
B_TIME_CYCLE_AGE,-4.629965,1.615994,-2.865088,4.168939e-03
B_TIME_DRIVE,-6.229242,0.472112,-13.194406,0.000000e+00
B_TIME_DRIVE_AGE,-2.260952,1.677081,-1.348147,1.776110e-01
B_TIME_PT,-2.292634,0.396540,-5.781592,7.399678e-09
B_TIME_PT_AGE,-3.078547,0.861126,-3.575023,3.501968e-04


# Market Share Stuff

In [5]:
data_filtered = df

populations = {
    'female_45_less': 2841376,
    'female_45_or_more': 1519948,
    'male_45_less': 2929408,
    'male_45_or_more': 1379198,
}

total_pop = sum(populations.values())

filters = {
    'male_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 0),
    'male_45_less': (data_filtered.age < 45) & (data_filtered.female == 0),
    'female_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 1),
    'female_45_less': (data_filtered.age < 45) & (data_filtered.female == 1),
}

sample_segments = {
    segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
}
print(sample_segments)

total_sample = sum(sample_segments.values())
print(f'Sample size: {total_sample}')

weights = {
    segment_name: populations[segment_name] * total_sample / (segment_size * total_pop)
    for segment_name, segment_size in sample_segments.items()
}
print(weights)

from biogeme.biogeme import BIOGEME
from biogeme.expressions import Beta, Variable, log, exp

from biogeme import models

for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]


prob_walk = models.logit(V, None, 1)
prob_cycle = models.logit(V, None, 2)
prob_pt = models.logit(V, None, 3)
prob_car = models.logit(V, None, 4)


weight = Variable('weight')
simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
    'Revenues PT': prob_pt * cost_transit,
}


{'male_45_or_more': np.int64(896), 'male_45_less': np.int64(1442), 'female_45_or_more': np.int64(984), 'female_45_less': np.int64(1678)}
Sample size: 5000
{'male_45_or_more': np.float64(0.8877139043468962), 'male_45_less': np.float64(1.1715720875375348), 'female_45_or_more': np.float64(0.890816074423909), 'female_45_less': np.float64(0.9765425353056789)}


# Increased Fuel Cost

In [6]:
data_filtered_increasedFuel = data_filtered.copy(deep=True)
data_filtered_increasedFuel["cost_driving"] += 1.5
database = db.Database('london', data_filtered_increasedFuel)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')

results_m4.bootstrap_samples = 100
results_bootstrapping = model_4.estimate(run_bootstrap=True)

betas = model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)
      
display(left)

display(right)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.336988,0.631595,2.419989e-03,0.028996,0.000000
1,0.976543,0.160315,0.613116,1.634623e-01,0.063107,0.480944
2,0.890816,0.228888,0.750883,1.643784e-06,0.020228,0.686665
3,0.887714,0.372489,0.574308,3.898100e-03,0.049305,0.000000
4,0.890816,0.186072,0.282248,5.003035e-01,0.031377,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.390542,0.563321,7.339913e-03,0.038797,0.585813
4996,1.171572,0.479774,0.424260,2.544727e-02,0.070518,0.719661
4997,0.976543,0.264772,0.335473,3.469903e-01,0.052765,0.397159
4998,0.976543,0.332862,0.648991,6.135947e-05,0.018086,0.499293


Market share for pt: 39.1%
Market share for car: 38.3%
Market share for walk: 18.8%
Market share for cycling: 3.7%


100%|██████████| 100/100 [00:34<00:00,  2.94it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.312394,0.604828,1.148729e-03,0.023647,0.000000
1,0.976543,0.149265,0.589186,1.429833e-01,0.052554,0.447794
2,0.890816,0.208152,0.731965,5.010303e-07,0.014803,0.624456
3,0.887714,0.346923,0.552132,2.190751e-03,0.042220,0.000000
4,0.890816,0.173378,0.258627,4.724354e-01,0.026071,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.368955,0.545634,4.859324e-03,0.033641,0.553433
4996,1.171572,0.459077,0.404849,1.854668e-02,0.060998,0.688615
4997,0.976543,0.252061,0.317723,3.254766e-01,0.044407,0.378091
4998,0.976543,0.309967,0.628247,3.069202e-05,0.015032,0.464950


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.363387,0.658318,4.953276e-03,0.034842,0.000000
1,0.976543,0.173460,0.631979,1.873742e-01,0.070634,0.520379
2,0.890816,0.246307,0.775243,6.601949e-06,0.028141,0.738922
3,0.887714,0.393966,0.601275,6.885044e-03,0.057301,0.000000
4,0.890816,0.204854,0.305815,5.268097e-01,0.037760,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.407504,0.582504,1.105559e-02,0.043180,0.611257
4996,1.171572,0.497633,0.443869,3.685047e-02,0.077583,0.746449
4997,0.976543,0.275560,0.352622,3.696299e-01,0.059776,0.413340
4998,0.976543,0.351871,0.671438,1.774418e-04,0.022004,0.527806


In [7]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)

left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

revenues_pt = simulated_values['Weighted revenues PT'].mean()
revenues_pt_left = left['Weighted revenues PT'].mean()
revenues_pt_right = right['Weighted revenues PT'].mean()
    
print(f"Revenue per person public transport: {revenues_pt:.3f} "
      f"CI: [{revenues_pt_left:.3f}-{revenues_pt_right:.3f}]")

print(f"Total Revenue public transport: {revenues_pt*total_pop:.3f} "
      f"CI: [{revenues_pt_left*total_pop:.3f}-{revenues_pt_right*total_pop:.3f}]")

Market share for pt: 39.1% CI: [36.8%-41.3%]
Market share for car: 38.3% CI: [36.0%-40.9%]
Market share for walk: 18.8% CI: [17.4%-20.2%]
Market share for cycling: 3.7% CI: [3.0%-4.4%]
Revenue per person public transport: 0.781 CI: [0.744-0.812]
Total Revenue public transport: 6768921.882 CI: [6446123.306-7041215.134]


# Decreased Transportation Cost

In [8]:
data_filtered_loweredTransit = data_filtered.copy(deep=True)
data_filtered_loweredTransit["cost_transit"] *= 0.8
database = db.Database('london', data_filtered_loweredTransit)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)


market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')

results_m4.bootstrap_samples = 100
results_bootstrapping = model_4.estimate(run_bootstrap=True)

betas = model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)
    
display(left)

display(right)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.281948,0.691767,2.024732e-03,0.024260,0.000000
1,0.976543,0.147838,0.664564,1.353458e-01,0.052252,0.354812
2,0.890816,0.202623,0.781298,1.306540e-06,0.016078,0.486295
3,0.887714,0.316337,0.638480,3.310465e-03,0.041872,0.000000
4,0.890816,0.171142,0.339839,4.601601e-01,0.028859,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.344691,0.616725,6.138458e-03,0.032446,0.413629
4996,1.171572,0.437362,0.479744,2.198121e-02,0.060913,0.524834
4997,0.976543,0.249857,0.392690,3.102722e-01,0.047181,0.299828
4998,0.976543,0.288170,0.696943,5.033539e-05,0.014837,0.345804


Market share for pt: 36.6%
Market share for car: 42.8%
Market share for walk: 17.3%
Market share for cycling: 3.2%


100%|██████████| 100/100 [00:47<00:00,  2.12it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.259627,0.670689,1.148553e-03,0.018757,0.000000
1,0.976543,0.131778,0.641086,1.196935e-01,0.044968,0.316266
2,0.890816,0.185259,0.762900,4.896358e-07,0.011192,0.444622
3,0.887714,0.297258,0.619174,2.117909e-03,0.035111,0.000000
4,0.890816,0.160657,0.323048,4.342681e-01,0.023744,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.327964,0.598878,4.405261e-03,0.028311,0.393557
4996,1.171572,0.417425,0.461615,1.655830e-02,0.052823,0.500910
4997,0.976543,0.237692,0.375838,2.921334e-01,0.040784,0.285231
4998,0.976543,0.270196,0.676483,2.530286e-05,0.012359,0.324235


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.302130,0.710468,3.849155e-03,0.029238,0.000000
1,0.976543,0.160993,0.685999,1.516187e-01,0.060549,0.386384
2,0.890816,0.222527,0.800238,4.518981e-06,0.020795,0.534065
3,0.887714,0.336595,0.654974,5.350256e-03,0.048577,0.000000
4,0.890816,0.184420,0.362078,4.802196e-01,0.034623,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.361807,0.631508,8.976045e-03,0.036814,0.434168
4996,1.171572,0.455860,0.497414,2.973469e-02,0.069840,0.547032
4997,0.976543,0.264030,0.406822,3.265178e-01,0.055368,0.316837
4998,0.976543,0.307603,0.715389,1.206520e-04,0.017179,0.369123


In [9]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)

left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

revenues_pt = simulated_values['Weighted revenues PT'].mean()
revenues_pt_left = left['Weighted revenues PT'].mean()
revenues_pt_right = right['Weighted revenues PT'].mean()
    
print(f"Revenue per person public transport: {revenues_pt:.3f} "
      f"CI: [{revenues_pt_left:.3f}-{revenues_pt_right:.3f}]")

print(f"Total Revenue public transport: {revenues_pt*total_pop:.3f} "
      f"CI: [{revenues_pt_left*total_pop:.3f}-{revenues_pt_right*total_pop:.3f}]")

Market share for pt: 36.6% CI: [34.6%-38.7%]
Market share for car: 42.8% CI: [40.5%-45.1%]
Market share for walk: 17.3% CI: [16.2%-18.4%]
Market share for cycling: 3.2% CI: [2.7%-3.9%]
Revenue per person public transport: 0.601 CI: [0.573-0.628]
Total Revenue public transport: 5213713.401 CI: [4972004.653-5443377.112]


# Highest Total Revenue

In [10]:
database = db.Database('london', data_filtered)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')

results_m4.bootstrap_samples = 100
results_bootstrapping = model_4.estimate(run_bootstrap=True)

betas = model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)
      
display(left)

display(right)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.281948,0.691767,2.024732e-03,0.024260,0.000000
1,0.976543,0.134774,0.674752,1.374206e-01,0.053053,0.404323
2,0.890816,0.185773,0.797808,1.334149e-06,0.016417,0.557320
3,0.887714,0.316337,0.638480,3.310465e-03,0.041872,0.000000
4,0.890816,0.171142,0.339839,4.601601e-01,0.028859,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.332627,0.628078,6.251457e-03,0.033044,0.498941
4996,1.171572,0.424154,0.491006,2.249719e-02,0.062343,0.636232
4997,0.976543,0.239898,0.397903,3.143915e-01,0.047808,0.359847
4998,0.976543,0.277248,0.707636,5.110771e-05,0.015064,0.415872


Market share for pt: 35.7%
Market share for car: 43.5%
Market share for walk: 17.4%
Market share for cycling: 3.3%


100%|██████████| 100/100 [00:42<00:00,  2.33it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.262096,0.666680,9.998863e-04,0.019702,0.000000
1,0.976543,0.123063,0.651004,1.175303e-01,0.046714,0.369189
2,0.890816,0.168253,0.777364,3.856605e-07,0.012669,0.504759
3,0.887714,0.296925,0.616927,1.811760e-03,0.035714,0.000000
4,0.890816,0.160418,0.316721,4.322933e-01,0.024103,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.317556,0.607436,3.928591e-03,0.029139,0.476334
4996,1.171572,0.408880,0.470382,1.617203e-02,0.055054,0.613319
4997,0.976543,0.228886,0.375417,2.893030e-01,0.041606,0.343329
4998,0.976543,0.259769,0.688865,2.303091e-05,0.011824,0.389654


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.308641,0.712419,4.092497e-03,0.029576,0.000000
1,0.976543,0.146251,0.699585,1.548588e-01,0.061899,0.438752
2,0.890816,0.205105,0.815572,4.512439e-06,0.021733,0.615315
3,0.887714,0.338004,0.657923,5.866862e-03,0.048462,0.000000
4,0.890816,0.185415,0.362770,4.880975e-01,0.035410,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.350628,0.645143,9.734341e-03,0.037751,0.525942
4996,1.171572,0.445640,0.509026,3.043639e-02,0.069899,0.668460
4997,0.976543,0.254631,0.419916,3.381521e-01,0.055802,0.381946
4998,0.976543,0.294293,0.724011,1.090705e-04,0.018492,0.441439


In [11]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)

left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

revenues_pt = simulated_values['Weighted revenues PT'].mean()
revenues_pt_left = left['Weighted revenues PT'].mean()
revenues_pt_right = right['Weighted revenues PT'].mean()
    
print(f"Revenue per person public transport: {revenues_pt:.3f} "
      f"CI: [{revenues_pt_left:.3f}-{revenues_pt_right:.3f}]")

print(f"Total Revenue public transport: {revenues_pt*total_pop:.3f} "
      f"CI: [{revenues_pt_left*total_pop:.3f}-{revenues_pt_right*total_pop:.3f}]")

Market share for pt: 35.7% CI: [33.6%-37.9%]
Market share for car: 43.5% CI: [41.1%-45.9%]
Market share for walk: 17.4% CI: [16.2%-18.7%]
Market share for cycling: 3.3% CI: [2.7%-4.0%]
Revenue per person public transport: 0.726 CI: [0.691-0.762]
Total Revenue public transport: 6297998.581 CI: [5991907.913-6603169.358]


# Value of Time

In [12]:
from biogeme.expressions import Derive
from biogeme.biogeme import BIOGEME
from biogeme.expressions import Beta, Variable, log, exp
from biogeme import models

database = db.Database('london', data_filtered)


for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]

weight = Variable('weight')
vot_pt = Derive(V_PT, 'dur_pt') / Derive(V_PT, 'cost_transit')
vot_car = Derive(V_DRIVE, 'dur_driving') / Derive(V_DRIVE, 'cost_driving')
simulate = {
    'weight': weight,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
}
biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())

print(-simulated_values['WTP PT time'].mean())
print(-simulated_values['WTP CAR time'].mean())

Parameter ASC_CYCLE not present in the model.
Parameter B_TIME_CYCLE not present in the model.
Parameter B_TIME_CYCLE_AGE not present in the model.
Parameter B_TIME_WALK not present in the model.
Parameter B_TIME_WALK_AGE not present in the model.


-20.057227395650173
-37.48980166043744


# Elasticities

In [13]:
prob_pt =  models.nested(V, None, nests, 3)  # Probability of public transport (3 is the index for PT in `V`)
prob_car = models.nested(V, None, nests, 4)  # Probability of driving (4 is the index for driving in `V`)
prob_walk = models.nested(V, None, nests, 1)  # Probability of walking (1 is the index for walking in `V`)
prob_cycle = models.nested(V, None, nests, 2)  # Probability of cycling (2 is the index for cycling in `V`)

direct_elas_car_cost = Derive(prob_car, 'cost_driving') * dur_driving / prob_car
direct_elas_pt_cost = Derive(prob_pt, 'cost_transit') * cost_transit / prob_pt

cross_elas_prob_car_pt_cost = Derive(prob_car, 'cost_transit') * cost_transit / prob_car
cross_elas_prob_walk_pt_cost = Derive(prob_walk, 'cost_transit') * cost_transit / prob_walk
cross_elas_prob_cycle_pt_cost = Derive(prob_cycle, 'cost_transit') * cost_transit / prob_cycle

cross_elas_prob_pt_car_cost = Derive(prob_pt, 'cost_driving') * cost_driving / prob_pt
cross_elas_prob_walk_car_cost = Derive(prob_walk, 'cost_driving') * cost_driving / prob_walk
cross_elas_prob_cycle_car_cost = Derive(prob_cycle, 'cost_driving') * cost_driving / prob_cycle

simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
    'direct_elas_pt_cost': direct_elas_pt_cost,
    'direct_elas_car_cost': direct_elas_car_cost,
    'cross_elas_prob_car_pt_cost': cross_elas_prob_car_pt_cost,
    'cross_elas_prob_walk_pt_cost': cross_elas_prob_walk_pt_cost,
    'cross_elas_prob_cycle_pt_cost': cross_elas_prob_cycle_pt_cost,
    'cross_elas_prob_pt_car_cost': cross_elas_prob_pt_car_cost,
    'cross_elas_prob_walk_car_cost': cross_elas_prob_walk_car_cost,
    'cross_elas_prob_cycle_car_cost': cross_elas_prob_cycle_car_cost
}

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())

simulated_values['numerator_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. pt']
    * simulated_values['direct_elas_pt_cost']
)

simulated_values['denominator_pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

agg_elast_pt_cost = (
    simulated_values['numerator_pt_cost'].sum()
    / simulated_values['denominator_pt'].sum()
)

print(f'Aggregate PT elasticity wrt cost: {agg_elast_pt_cost:.3g}')

simulated_values['numerator_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. car']
    * simulated_values['direct_elas_car_cost']
)

simulated_values['denominator_car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

agg_elast_car_cost = (
    simulated_values['numerator_car_cost'].sum()
    / simulated_values['denominator_car'].sum()
)

print(f'Aggregate CAR elasticity wrt cost: {agg_elast_car_cost:.3g}')

simulated_values['numerator_cross_car_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. car']
    * simulated_values['cross_elas_prob_car_pt_cost']
)

agg_elast_pt_cost = (
    simulated_values['numerator_cross_car_pt_cost'].sum()
    / simulated_values['denominator_car'].sum()
)

print(f'Cross CAR elasticity wrt PT cost: {agg_elast_pt_cost:.3g}')

simulated_values['numerator_cross_walk_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. walk']
    * simulated_values['cross_elas_prob_walk_pt_cost']
)

simulated_values['denominator_walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)

agg_elast_cross_walk_pt_cost = (
    simulated_values['numerator_cross_walk_pt_cost'].sum()
    / simulated_values['denominator_walk'].sum()
)

print(f'Cross WALK elasticity wrt PT cost: {agg_elast_cross_walk_pt_cost:.3g}')

simulated_values['numerator_cross_cycle_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. cycle']
    * simulated_values['cross_elas_prob_cycle_pt_cost']
)

simulated_values['denominator_cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

agg_elast_cross_cycle_pt_cost = (
    simulated_values['numerator_cross_cycle_pt_cost'].sum()
    / simulated_values['denominator_cycle'].sum()
)

print(f'Cross CYCLE elasticity wrt PT cost: {agg_elast_cross_cycle_pt_cost:.3g}')


# NOW FOR CAR TIME

simulated_values['numerator_cross_pt_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. pt']
    * simulated_values['cross_elas_prob_pt_car_cost']
)

agg_elast_cross_pt_car_cost = (
    simulated_values['numerator_cross_pt_car_cost'].sum()
    / simulated_values['denominator_pt'].sum()
)

print(f'Cross PT elasticity wrt CAR cost: {agg_elast_cross_pt_car_cost:.3g}')

simulated_values['numerator_cross_walk_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. walk']
    * simulated_values['cross_elas_prob_walk_car_cost']
)

agg_elast_cross_walk_car_cost = (
    simulated_values['numerator_cross_walk_car_cost'].sum()
    / simulated_values['denominator_walk'].sum()
)

print(f'Cross WALK elasticity wrt CAR cost: {agg_elast_cross_walk_car_cost:.3g}')

simulated_values['numerator_cross_cycle_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. walk']
    * simulated_values['cross_elas_prob_walk_car_cost']
)

agg_elast_cross_walk_car_cost = (
    simulated_values['numerator_cross_walk_car_cost'].sum()
    / simulated_values['denominator_walk'].sum()
)

print(f'Cross WALK elasticity wrt CAR cost: {agg_elast_cross_walk_car_cost:.3g}')

NameError: name 'nests' is not defined